# Import Dependencies

In [1]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
import math
import os

# Function to calculate the distance between two bounding boxes

In [2]:
# Distance between 2 bounding boxes
def calculate_distance(box1, box2):
    # Extract coordinates from each bounding box
    x1, y1, _, _ = box1
    x2, y2, _, _ = box2

    # Calculate Euclidean distance between the centers of the bounding boxes
    distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    
    return distance

# Function to calculate the area of the bounding box

In [3]:
from bbox import BBox2D
def calculate_area(box):
    x1, y1, x2, y2 = box
    return (math.dist([x1, y2], [x1, y1]) * math.dist([x1, y2], [x2, y2]))

# Function to print the warning

In [4]:
def wrn_msg():
    root = tk.Tk()
    root.withdraw()
    messagebox.showinfo("Warning", "Too Close!")

In [5]:
def wrn_msg2():
    root = tk.Tk()
    root.withdraw()
    messagebox.showinfo("Warning", "Slow down!")

# Using a YOLOv8 model

In [6]:
# Get the pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')

# Using OpenCV to read the frames and perform object detection

In [7]:
# Import Video
cap = cv2.VideoCapture(os.path.join('Input', 'Video1.mp4'))

# Detection for each frame 
for frame_index in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = cap.read()
    results = model.predict(frame, save=True, conf = 0.5, classes = [0, 1, 2, 3, 5, 6, 7])
    for r in results:
        annotator = Annotator(frame)
        m = 0
        for box in r.boxes:
            b = box.xyxy[0]  # get box coordinates in (left, top, right, bottom) format
            c = box.cls
            m+=1
            annotator.box_label(b, model.names[int(c)])

        mat = np.zeros((m, m))
        res = frame.shape[0] * frame.shape[1]
        areas = []
        x, y = 0, 0
        for i in r.boxes:
            box1 = i.xyxy[0]
            y = 0
            areas.append(calculate_area(box1))
            for j in r.boxes:
                box2 = j.xyxy[0]
                dist = calculate_distance(box1, box2)
                mat[x][y] = dist
                y+=1
            x+=1

    imag = annotator.result()
    
    # Checking the distance between the cars
    d = [d for row in mat for d in row]
    
    # Displaying the video
    cv2.imshow('YOLO V8 Detection', imag) 

    for x in d:
        if x < 50:
            if x != 0:
                wrn_msg()
                break

    for area in areas:
        if (area/res) > 0.1:
            wrn_msg2()
            areas = []
            break


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 car, 1 truck, 215.1ms
Speed: 8.0ms preprocess, 215.1ms inference, 7.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict2

0: 384x640 1 car, 1 truck, 119.0ms
Speed: 6.0ms preprocess, 119.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict2

0: 384x640 1 car, 1 truck, 168.0ms
Speed: 5.0ms preprocess, 168.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict2

0: 384x640 2 cars, 1 truck, 114.0ms
Speed: 5.0ms preprocess, 114.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict2

0: 384x640 1 car, 1 bus, 91.0ms
Speed: 5.0ms preprocess, 91.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict2

0: 384x640 1 car, 2 trucks, 118.0ms
Speed: 3.0ms preprocess, 118.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results 